# AML Assignment 3

### Step 1 :Importing all required packages

In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model,svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Performing grid search
import xgboost as xgb

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### Step 2 :Getting the dataset for test and train

In [2]:
## Getting the test and train data
Test_Data=pd.read_table("D:/Sem 1 IU My Stuff/AML/Assignment 3/a3-test.data",sep=" ",header=None)
Test_labels = pd.read_csv("D:/Sem 1 IU My Stuff/AML/Assignment 3/a3-test.labels.txt", sep="\t", header=None)
Train_Data=pd.read_table("D:/Sem 1 IU My Stuff/AML/Assignment 3/a3-train.data",sep=" ",header=None)
Train_labels = pd.read_csv("D:/Sem 1 IU My Stuff/AML/Assignment 3/a3-train.labels.txt", sep="\t", header=None)

Test_Data["Labels"]=Test_labels
Test_Data=Test_Data.drop(Test_Data.columns[500], axis=1)

Train_Data["Labels"]=Train_labels
Train_Data=Train_Data.drop(Train_Data.columns[500], axis=1)

### Step 3 : EDA on the dataset

#### 3.a) Checking for NA 

In [3]:
print("Presence of Null values in Training Data : ",Train_Data.isnull().values.any())
print("Presence of Null values in Test Data : ",Test_Data.isnull().values.any())

Presence of Null values in Training Data :  False
Presence of Null values in Test Data :  False


#### 3.b) Checking for Distribution of Labels

In [4]:
Train_Data.groupby('Labels').count()[1]

Labels
-1    1000
 1    1000
Name: 1, dtype: int64

In [5]:
Test_Data.groupby('Labels').count()[1]

Labels
-1    300
 1    300
Name: 1, dtype: int64

#### 3.c)Checking for variable distribution

In [6]:
des=Train_Data.describe().T
print(des.to_string())

         count      mean         std    min     25%    50%     75%    max
0       2000.0  481.7225    6.421769  462.0  477.00  482.0  486.00  503.0
1       2000.0  483.4525   30.186294  381.0  464.00  483.0  503.00  600.0
2       2000.0  510.1660   38.899165  370.0  485.00  510.5  536.00  654.0
3       2000.0  483.3845    9.059895  453.0  477.00  483.0  490.00  519.0
4       2000.0  501.6125   41.389418  371.0  475.00  500.0  528.00  688.0
5       2000.0  479.2590    6.795956  459.0  475.00  479.0  484.00  505.0
6       2000.0  480.1095   40.575925  334.0  452.75  480.0  506.25  611.0
7       2000.0  476.5650    1.384461  471.0  476.00  477.0  477.00  481.0
8       2000.0  486.7935   15.043836  430.0  477.00  487.0  496.25  536.0
9       2000.0  478.7890    7.190920  455.0  474.00  479.0  484.00  503.0
10      2000.0  486.5175   36.556831  354.0  462.00  486.0  511.00  620.0
11      2000.0  490.6535   28.198404  389.0  472.00  491.0  508.25  603.0
12      2000.0  478.1645   34.169434  

#### 3.d)Unique between the columns in python

In [7]:
#des=pd.DataFrame(Train_Data.corr())
uni=Train_Data.T.apply(lambda x: x.nunique(), axis=1)
print(uni.to_string())

0          40
1         170
2         208
3          60
4         222
5          44
6         213
7          11
8          92
9          47
10        196
11        160
12        183
13        227
14         76
15        130
16         80
17        169
18        130
19         64
20         81
21        126
22         44
23         63
24        227
25        157
26        125
27         75
28         75
29        133
30         52
31        194
32        194
33        211
34        141
35        219
36         72
37        136
38         32
39         12
40         24
41        209
42        130
43         97
44        160
45        137
46        196
47        183
48        200
49        146
50        229
51        197
52        124
53         34
54        160
55        122
56        218
57        186
58        217
59        122
60        112
61         43
62        205
63         44
64        402
65        197
66        199
67        128
68         34
69         45
70        224
71    

## Baseline Model 1 : Logistic Regression 

In [8]:
clf = LogisticRegression(fit_intercept=True,C=1)
clf.fit(Train_Data.iloc[:,:499], Train_Data.iloc[:,-1])

LR_Accuracy_Test=clf.score(Test_Data.iloc[:,:499], Test_Data.iloc[:,-1])
LR_Accuracy_train=clf.score(Train_Data.iloc[:,:499], Train_Data.iloc[:,-1])

Logistic_Prediction_Train=clf.predict_proba(Train_Data.iloc[:,:499])[:,1]
Logistic_Prediction_Test=clf.predict_proba(Test_Data.iloc[:,:499])[:,1]

print("Accuracy of Train Data : ",LR_Accuracy_train)
print("Accuracy of Test Data : ",LR_Accuracy_Test)


Accuracy of Train Data :  0.741
Accuracy of Test Data :  0.5866666666666667


## Baseline Model 2 : Random Forest Model 

In [9]:
RF_Model_Base = RandomForestClassifier()
RF_Model_Base.fit(Train_Data.iloc[:,:499], Train_Data.iloc[:,-1])

RF_predictions_Test = RF_Model_Base.predict(Test_Data.iloc[:,:499])
RF_predictions_Train = RF_Model_Base.predict(Train_Data.iloc[:,:499])

Train_Accuracy_RF=RF_Model_Base.score(Train_Data.iloc[:,:499], Train_Data.iloc[:,-1])
Test_Accuracy_RF=RF_Model_Base.score(Test_Data.iloc[:,:499], Test_Data.iloc[:,-1])

print("Accuracy of Train Data : ",Train_Accuracy_RF)
print("Accuracy of Test Data : ",Test_Accuracy_RF)

Accuracy of Train Data :  0.9875
Accuracy of Test Data :  0.6366666666666667


## Baseline Model 3 : KNN 

In [10]:
#making the instance
KNN_Base = KNeighborsClassifier()

#Learning
KNN_Base.fit(Train_Data.iloc[:,:499], Train_Data.iloc[:,-1])
KNN_Accuracy_Test=KNN_Base.score(Test_Data.iloc[:,:499], Test_Data.iloc[:,-1])
KNN_Accuracy_train=KNN_Base.score(Train_Data.iloc[:,:499], Train_Data.iloc[:,-1])

print("Accuracy of Train Data : ",KNN_Accuracy_train)
print("Accuracy of Test Data : ",KNN_Accuracy_Test)


Accuracy of Train Data :  0.8265
Accuracy of Test Data :  0.6966666666666667


## Baseline Model 4 : Gradient Boosting

In [11]:
#making the instance
GB_model_Base = GradientBoostingClassifier()

#Learning
GB_model_Base.fit(Train_Data.iloc[:,:499], Train_Data.iloc[:,-1])
GB_Accuracy_Test=GB_model_Base.score(Test_Data.iloc[:,:499], Test_Data.iloc[:,-1])
GB_Accuracy_train=GB_model_Base.score(Train_Data.iloc[:,:499], Train_Data.iloc[:,-1])

print("Accuracy of Train Data : ",GB_Accuracy_train)
print("Accuracy of Test Data : ",GB_Accuracy_Test)


Accuracy of Train Data :  0.962
Accuracy of Test Data :  0.7466666666666667


## Feature Engineering

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFECV

ImpFea = RandomForestClassifier()
ImpFea_vf = RFECV(ImpFea)
ImpFea_vff = ImpFea_vf.fit_transform(Train_Data.iloc[:,:499], Train_Data.iloc[:,-1])
Columns_to_Retained = Train_Data.iloc[:,:499].columns[ImpFea_vf.get_support()].values


In [13]:
Columns_to_Retained

array([28, 48, 64, 105, 128, 153, 241, 281, 318, 336, 338, 378, 433, 442,
       451, 453, 472, 475, 493], dtype=object)

In [14]:
X_train=pd.DataFrame(Train_Data[Columns_to_Retained])
Y_train=Train_Data.iloc[:,-1]
X_test=Test_Data[Columns_to_Retained]
Y_test=Test_Data.iloc[:,-1]

## Model Tuning (Grid Search with select Features)

### Logistic Regression 2

In [15]:
C_value = [0.0001,0.0002,0.0003,0.0004,0.0005,0.0006,0.0007,0.0008,0.0009,0.001,0.01,0.1,1]

grid={"C":C_value, "penalty":["l2","l1"]}

Logistic=LogisticRegression()
Log_GS=GridSearchCV(Logistic,grid,cv=3)
Log_GS.fit(X_train,Y_train)

print("Best parameters after Grid Search ",Log_GS.best_params_)

LR_Accuracy_Test=Log_GS.score(X_test,Y_test)
LR_Accuracy_train=Log_GS.score(X_train,Y_train)

Logistic_Prediction_Train=Log_GS.predict_proba(X_train)[:,1]
Logistic_Prediction_Test=Log_GS.predict_proba(X_test)[:,1]

print("Accuracy on Train Data for Logistic Regression after Grid Search: ",LR_Accuracy_train)
print("Accuracy on Test Data for Logistic Regression after Grid Search:: ",LR_Accuracy_Test)


Best parameters after Grid Search  {'C': 0.0009, 'penalty': 'l1'}
Accuracy on Train Data for Logistic Regression after Grid Search:  0.6165
Accuracy on Test Data for Logistic Regression after Grid Search::  0.5916666666666667


### Random Forest 

In [16]:
RF = RandomForestClassifier()

param_grid = {'n_estimators': [300,400, 500,600,700,800],
    'max_features': ['auto', 'sqrt', 'log2'],
    'criterion' :['gini', 'entropy']}

# Create a based model

# Instantiate the grid search model
RF_Grid = GridSearchCV(estimator = RF, param_grid = param_grid,cv = 3, n_jobs = -1, verbose = 2)
RF_Grid.fit(X_train,Y_train)

print("The best parameters are",RF_Grid.best_params_)

RF_Accuracy_Test=RF_Grid.score(X_test,Y_test)
RF_Accuracy_train=RF_Grid.score(X_train,Y_train)

RF_Prediction_Train=RF_Grid.predict_proba(X_train)[:,1]
RF_Prediction_Test=RF_Grid.predict_proba(X_test)[:,1]

print("Accuracy on Train Data for Random Forest after Grid Search: ",RF_Accuracy_train)
print("Accuracy on Test Data for Random Forest after Grid Search:: ",RF_Accuracy_Test)


Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:  2.5min finished


The best parameters are {'criterion': 'entropy', 'max_features': 'auto', 'n_estimators': 700}
Accuracy on Train Data for Random Forest after Grid Search:  1.0
Accuracy on Test Data for Random Forest after Grid Search::  0.895


### KNN 

In [17]:
model = KNeighborsClassifier(n_jobs=-1)

params = {'n_neighbors':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25],
          'leaf_size':[1,2,3,5],
          'weights':['uniform', 'distance']
        }

KNN_Grid = GridSearchCV(model, param_grid=params,cv=3, n_jobs=1)

KNN_Grid.fit(X_train,Y_train)

print("Best Hyper Parameters:\n",KNN_Grid.best_params_)

KNN_Accuracy_Test=KNN_Grid.score(X_test,Y_test)
KNN_Accuracy_train=KNN_Grid.score(X_train,Y_train)

KNN_Prediction_Train=KNN_Grid.predict_proba(X_train)[:,1]
KNN_Prediction_Test=KNN_Grid.predict_proba(X_test)[:,1]

print("Accuracy on Train Data for KNN after Grid Search : ",KNN_Accuracy_train)
print("Accuracy on Test Data for KNN after Grid Search : ",KNN_Accuracy_Test)


Best Hyper Parameters:
 {'leaf_size': 1, 'n_neighbors': 4, 'weights': 'distance'}
Accuracy on Train Data for KNN after Grid Search :  1.0
Accuracy on Test Data for KNN after Grid Search :  0.8916666666666667


In [58]:
parameters = {
    "learning_rate": [0.05, 0.1, 0.25,  0.75, 1,0.5],
    "max_depth":[8,10,15,20,40],
    "n_estimators":[10,100,150]
    }

GBM = GridSearchCV(GradientBoostingClassifier(), parameters, cv=5, n_jobs=-1)

GBM.fit(X_train,Y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'learning_rate': [0.05, 0.1, 0.25, 0.75, 1, 0.5], 'max_depth': [8, 10, 15, 20, 40], 'n_estimators': [10, 100, 150]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [59]:
GBM_Accuracy_Test=GBM.score(X_test,Y_test)
GBM_Accuracy_train=GBM.score(X_train,Y_train)

GB_Prediction_Train=GBM.predict_proba(X_train)[:,1]
GB_Prediction_Test=GBM.predict_proba(X_test)[:,1]

print("Best Hyper Parameters:\n",GBM.best_params_)
print("Accuracy on Train Data for GBM after Grid Search : ",GBM_Accuracy_train)
print("Accuracy on Test Data for GBM after Grid Search : ",GBM_Accuracy_Test)


Best Hyper Parameters:
 {'learning_rate': 0.25, 'max_depth': 8, 'n_estimators': 100}
Accuracy on Train Data for GBM after Grid Search :  1.0
Accuracy on Test Data for GBM after Grid Search :  0.8766666666666667


In [60]:
## Stacking 
Stacking_data_Train=pd.DataFrame( {'Logistic': Logistic_Prediction_Train,'KNN': KNN_Prediction_Train,'RandomForest':RF_Prediction_Train,'GBM':GB_Prediction_Train})
Stacking_data_Test=pd.DataFrame( {'Logistic': Logistic_Prediction_Test,'KNN': KNN_Prediction_Test,'RandomForest':RF_Prediction_Test,'GBM':GB_Prediction_Test})

In [61]:
Stacking_data_Train.corr()

,Logistic,KNN,RandomForest,GBM
Logistic,1.000000,0.247527,0.290880,0.247792
KNN,0.247527,1.000000,0.984654,0.999999
RandomForest,0.290880,0.984654,1.000000,0.984812
GBM,0.247792,0.999999,0.984812,1.000000


In [62]:
xgb_model = xgb.XGBClassifier()
parameters = {'nthread':[4],'objective':['binary:logistic'],'learning_rate': [0.05,0.1,0.5],
              #'min_child_weight': [11],
              #'silent': [1],
              #'subsample': [0.8],
              #'colsample_bytree': [0.7],
              'n_estimators': [100,500,700], 
              'seed': [1]}


Stacking = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=4, 
                   scoring='accuracy',
                   verbose=2, refit=True)

Stacking.fit(Stacking_data_Train, Train_Data.iloc[:,-1])

Fitting 4 folds for each of 9 candidates, totalling 36 fits


[Parallel(n_jobs=5)]: Done  36 out of  36 | elapsed:   40.8s finished


GridSearchCV(cv=4, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=5,
       param_grid={'nthread': [4], 'objective': ['binary:logistic'], 'learning_rate': [0.05, 0.1, 0.5], 'n_estimators': [100, 500, 700], 'seed': [1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=2)

In [52]:
Stacking

GridSearchCV(cv=4, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=5,
       param_grid={'nthread': [4], 'objective': ['binary:logistic'], 'learning_rate': [0.05, 0.1, 0.5], 'n_estimators': [100, 500, 700], 'seed': [1]},
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=2)

In [63]:
metrics.accuracy_score(Test_Data.iloc[:,-1], Stacking.predict(Stacking_data_Test))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8916666666666667